# Data extention for Ace_Bikes_Data.xlsx

In [1]:
# imports
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

In [2]:
# Set random seed for reproducibility
np.random.seed(1234)

In [3]:
# Load main Excel file
df_main = pd.read_excel('../data_original/Ace_Bikes_Data.xlsx', sheet_name='Data')

# Load new employees
df_new_employees = pd.read_excel('../data_new/newEmployees.xlsx')

# Load new line item sales
df_new_line_items = pd.read_csv('../data_new/newLineItemSales.csv')


In [4]:
# Find where each table starts based on column headers
# EmployeeDates: columns 0-3 (EmployeeID, StartDate, TerminationDate, LocationID)
# EmployeeReviewInfo: columns 5-11 (EmpID, Date, Salesmanship, ProductKnowledge, TeamPlayer, Innovator, Satisfaction)
# EmployeeTerminationReasons: columns 13-14 (EmployeeID, Reason)
# LineItemReturns: columns 40-41 (LineItemID, ReturnID)

# Find the last row with data for each table
employee_dates_last_row = df_main['EmployeeID'].notna().sum() - 1
employee_review_last_row = df_main['EmpID'].notna().sum() - 1
employee_termination_last_row = df_main['EmployeeID.1'].notna().sum() - 1
line_item_returns_last_row = df_main['LineItemID'].notna().sum() - 1

print(f"EmployeeDates ends at row: {employee_dates_last_row}")
print(f"EmployeeReviewInfo ends at row: {employee_review_last_row}")
print(f"EmployeeTerminationReasons ends at row: {employee_termination_last_row}")
print(f"LineItemReturns ends at row: {line_item_returns_last_row}")

EmployeeDates ends at row: 54
EmployeeReviewInfo ends at row: 205
EmployeeTerminationReasons ends at row: 21
LineItemReturns ends at row: 3538


In [5]:
# APPEND NEW EMPS

# Extract only the required columns
new_employee_dates = df_new_employees[['EmployeeID', 'StartDate', 'TerminationDate', 'LocationID']].copy()

print(f"\nAppending {len(new_employee_dates)} employee records to EmployeeDates table")

# Add new employee dates starting from the next available row
start_row = employee_dates_last_row + 1
for idx, row in new_employee_dates.iterrows():
    df_main.loc[start_row + idx, 'EmployeeID'] = row['EmployeeID']
    df_main.loc[start_row + idx, 'StartDate'] = row['StartDate']
    df_main.loc[start_row + idx, 'TerminationDate'] = row['TerminationDate']
    df_main.loc[start_row + idx, 'LocationID'] = row['LocationID']

print(f"✓ Added {len(new_employee_dates)} employee date records")


Appending 26 employee records to EmployeeDates table
✓ Added 26 employee date records


In [6]:
# GENERATE EMP REVIEWS

def generate_review_dates(start_date, end_date=None):
    """Generate semi-annual review dates (January and July)"""
    if pd.isna(start_date):
        return []
    
    start = pd.to_datetime(start_date)
    
    # If no end date, use current date (2023-12-31 as a reasonable end)
    if pd.isna(end_date):
        end = pd.to_datetime('2023-12-31')
    else:
        end = pd.to_datetime(end_date)
    
    review_dates = []
    
    # Start from the first review period after start date
    # Reviews are in January (around 10th) and July (around 2nd-8th)
    current_year = start.year
    
    # Check if started before July, add July review
    if start <= pd.to_datetime(f'{current_year}-07-01'):
        july_review = pd.to_datetime(f'{current_year}-07-{np.random.randint(2, 9)}')
        if july_review >= start and july_review <= end:
            review_dates.append(july_review)
    
    # Move to next January
    current_year += 1
    
    while True:
        # January review
        jan_review = pd.to_datetime(f'{current_year}-01-{np.random.randint(9, 17)}')
        if jan_review <= end:
            review_dates.append(jan_review)
        else:
            break
        
        # July review
        july_review = pd.to_datetime(f'{current_year}-07-{np.random.randint(1, 9)}')
        if july_review <= end:
            review_dates.append(july_review)
        else:
            break
        
        current_year += 1
    
    return review_dates


In [7]:
# generate rating scores

def generate_rating(has_training, base_mean=3.5, base_std=0.8):
    """
    Generate a rating score (1-5).
    If employee has training, boost the mean rating.
    """
    if has_training:
        # Training increases mean by 0.5-0.8 points
        mean = base_mean + np.random.uniform(0.5, 0.8)
    else:
        mean = base_mean
    
    # Generate rating with some randomness
    rating = np.random.normal(mean, base_std)
    
    # Clip to valid range and round to 1 decimal
    rating = np.clip(rating, 2.0, 5.0)
    return round(rating, 1)

In [8]:
# Generate reviews for each new employee
new_reviews = []

for _, emp in df_new_employees.iterrows():
    emp_id = emp['EmployeeID']
    start_date = emp['StartDate']
    termination_date = emp['TerminationDate']
    
    # Get training flags
    skills_training = emp['SkillsTraining']
    salesmanship_training = emp['SalesmanshipTraining']
    product_training = emp['ProductTraining']
    
    # Generate review dates
    review_dates = generate_review_dates(start_date, termination_date)
    
    # Generate reviews for each date
    for review_date in review_dates:
        review = {
            'EmpID': emp_id,
            'Date': review_date,
            'Salesmanship': generate_rating(salesmanship_training, base_mean=3.4),
            'ProductKnowledge': generate_rating(product_training, base_mean=3.6),
            'TeamPlayer': generate_rating(skills_training, base_mean=3.7),
            'Innovator': generate_rating(skills_training, base_mean=3.5),
            'Satisfaction': generate_rating(skills_training, base_mean=3.3)
        }
        new_reviews.append(review)

df_new_reviews = pd.DataFrame(new_reviews)

In [9]:
print(f"\nGenerated {len(df_new_reviews)} review records for {len(df_new_employees)} employees")
print(f"Average reviews per employee: {len(df_new_reviews) / len(df_new_employees):.1f}")

# Sample of generated reviews
# print("\nSample of generated reviews:")
df_new_reviews.head(10)


Generated 20 review records for 26 employees
Average reviews per employee: 0.8


,EmpID,Date,Salesmanship,ProductKnowledge,TeamPlayer,Innovator,Satisfaction
0,56,2022-07-05,4.5,3.9,3.4,3.2,3.0
1,56,2023-01-15,4.2,3.6,3.4,4.3,2.8
2,56,2023-07-06,3.9,3.1,4.4,2.8,3.1
3,57,2022-07-05,3.4,2.0,3.3,4.3,3.5
4,57,2023-01-12,4.8,3.2,4.8,4.4,2.9
5,57,2023-07-04,3.5,4.0,5.0,4.0,2.2
6,58,2022-07-06,3.7,4.7,4.3,4.3,4.7
7,58,2023-01-11,3.2,2.6,5.0,4.1,3.9
8,58,2023-07-08,4.2,4.1,4.1,4.0,4.8
9,59,2022-07-02,3.9,4.3,3.4,2.9,4.0


In [10]:
# now append to EmployeeReviewInfo table
start_row = employee_review_last_row + 1
for idx, row in df_new_reviews.iterrows():
    df_main.loc[start_row + idx, 'EmpID'] = row['EmpID']
    df_main.loc[start_row + idx, 'Date'] = row['Date']
    df_main.loc[start_row + idx, 'Salesmanship'] = row['Salesmanship']
    df_main.loc[start_row + idx, 'ProductKnowledge'] = row['ProductKnowledge']
    df_main.loc[start_row + idx, 'TeamPlayer'] = row['TeamPlayer']
    df_main.loc[start_row + idx, 'Innovator'] = row['Innovator']
    df_main.loc[start_row + idx, 'Satisfaction'] = row['Satisfaction']

print(f"✓ Added {len(df_new_reviews)} employee review records")

✓ Added 20 employee review records


In [11]:
# ADD TERMINATION REASON

# Find employees with termination dates
terminated_employees = df_new_employees[df_new_employees['TerminationDate'].notna()]

if len(terminated_employees) > 0:
    print(f"\nFound {len(terminated_employees)} terminated employees")
    
    # Define termination reasons with probabilities
    reasons = ['Another Job', 'Moved', 'Terminated']
    probabilities = [0.50, 0.25, 0.25]
    
    # Generate random reasons
    termination_records = []
    for _, emp in terminated_employees.iterrows():
        reason = np.random.choice(reasons, p=probabilities)
        termination_records.append({
            'EmployeeID': emp['EmployeeID'],
            'Reason': reason
        })
    
    df_terminations = pd.DataFrame(termination_records)
    
    # Append to EmployeeTerminationReasons table
    start_row = employee_termination_last_row + 1
    for idx, row in df_terminations.iterrows():
        df_main.loc[start_row + idx, 'EmployeeID.1'] = row['EmployeeID']
        df_main.loc[start_row + idx, 'Reason'] = row['Reason']
    
    print(f"✓ Added {len(df_terminations)} termination reason records")
    print("\nTermination reasons breakdown:")
    print(df_terminations['Reason'].value_counts())
else:
    print("\nNo terminated employees found - skipping termination reasons")



No terminated employees found - skipping termination reasons


In [12]:
# GEN LINE ITEM RETURNS

np.random.seed(1234)

# Sample 4% to 5% of line items
sample_rate = np.random.uniform(0.040, 0.050)
n_returns = int(len(df_new_line_items) * sample_rate)

print(f"\nSampling {sample_rate:.2%} of {len(df_new_line_items)} line items = {n_returns} returns")

# Randomly sample line items
sampled_line_items = df_new_line_items.sample(n=n_returns, random_state=42)


Sampling 4.19% of 82122 line items = 3442 returns


In [13]:
# Assign ReturnIDs with specified probabilities
return_ids = ['R1', 'R2', 'R3']
return_probabilities = [0.34, 0.34, 0.32]

returns_data = []
for _, item in sampled_line_items.iterrows():
    return_id = np.random.choice(return_ids, p=return_probabilities)
    returns_data.append({
        'LineItemID': item['LineItemID'],
        'ReturnID': return_id
    })

df_returns = pd.DataFrame(returns_data)

print(f"\nReturn ID distribution:")
print(df_returns['ReturnID'].value_counts().sort_index())
print(f"\nPercentages:")
print(df_returns['ReturnID'].value_counts(normalize=True).sort_index())

# Append to LineItemReturns table
start_row = line_item_returns_last_row + 1
for idx, row in df_returns.iterrows():
    df_main.loc[start_row + idx, 'LineItemID'] = row['LineItemID']
    df_main.loc[start_row + idx, 'ReturnID'] = row['ReturnID']

print(f"✓ Added {len(df_returns)} line item return records")


Return ID distribution:
ReturnID
R1    1154
R2    1171
R3    1117
Name: count, dtype: int64

Percentages:
ReturnID
R1    0.335270
R2    0.340209
R3    0.324521
Name: proportion, dtype: float64
✓ Added 3442 line item return records


In [14]:
# SAVE

# Create output directory
output_dir = '../data_new/'
os.makedirs(output_dir, exist_ok=True)

output_path = os.path.join(output_dir, 'newAce_Bikes_Data.xlsx')

# Save to Excel
print(f"\nSaving to: {output_path}")
df_main.to_excel(output_path, sheet_name='Data', index=False)

print(" File saved!")


Saving to: ../data_new/newAce_Bikes_Data.xlsx
 File saved!


In [15]:
# summary report

print(f"""
Employee Updates:
  • New employees added: {len(new_employee_dates)}
  • Employee reviews generated: {len(df_new_reviews)}
  • Termination reasons added: {len(terminated_employees)}

Line Item Updates:
  • Total line items: {len(df_new_line_items)}
  • Returns generated: {len(df_returns)} ({sample_rate:.2%})
  • Return distribution:
    - R1: {(df_returns['ReturnID'] == 'R1').sum()} ({(df_returns['ReturnID'] == 'R1').mean():.1%})
    - R2: {(df_returns['ReturnID'] == 'R2').sum()} ({(df_returns['ReturnID'] == 'R2').mean():.1%})
    - R3: {(df_returns['ReturnID'] == 'R3').sum()} ({(df_returns['ReturnID'] == 'R3').mean():.1%})

Output:
  • File: newAce_Bikes_Data.xlsx
  • Location: {output_dir}
""")


Employee Updates:
  • New employees added: 26
  • Employee reviews generated: 20
  • Termination reasons added: 0

Line Item Updates:
  • Total line items: 82122
  • Returns generated: 3442 (4.19%)
  • Return distribution:
    - R1: 1154 (33.5%)
    - R2: 1171 (34.0%)
    - R3: 1117 (32.5%)

Output:
  • File: newAce_Bikes_Data.xlsx
  • Location: ../data_new/

